In [1]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split

IPPR CNN Model 

Import image data from dataset containing 20000 positive images and 20000 negative images with each image being 227 x 227 pixels large. Kaggle link: https://www.kaggle.com/datasets/arnavr10880/concrete-crack-images-for-classification/data

In [12]:
data = tf.keras.utils.image_dataset_from_directory('Data', image_size=(227, 227), batch_size=64)

Found 40000 files belonging to 2 classes.


Shuffle the dataset before splitting the data into training, testing and validation sets.

In [13]:
shuffled_data = data.shuffle(buffer_size=100, reshuffle_each_iteration=False)

Splitting the dataset into training, testining and validation sets.

In [14]:
training_split_size = int(len(data)*.7)
testing_split_size = int(len(data)*.15)
validation_split_size = int(len(data)*.15)

In [18]:
training_set = shuffled_data.take(training_split_size)
testing_set = shuffled_data.skip(training_split_size).take(testing_split_size)
validation_set = shuffled_data.skip(training_split_size+testing_split_size).take(validation_split_size)

Model Building

In [ ]:
model = Sequential()

model.add(Conv2D(32, (21,21), strides = 2, activation = 'relu', input_shape=(227,227,3))) 
model.add(MaxPooling2D())
model.add(Conv2D(64, (15,15), strides = 1, activation = 'relu'))
model.add(MaxPooling2D())
